# Coding a Transformer from Scratch

### Cristiano De Nobili - My Contacts
For any questions or doubts you can find my contacts here:

<p align="center">

[<img src="https://img.freepik.com/premium-vector/linkedin-logo_578229-227.jpg?w=1060" width="25">](https://www.linkedin.com/in/cristiano-de-nobili/) [<img src="https://1.bp.blogspot.com/-Rwqcet_SHbk/T8_acMUmlmI/AAAAAAAAGgw/KD_fx__8Q4w/s1600/Twitter+bird.png" width="30">](https://twitter.com/denocris)        

</p>

or here (https://denocris.com).




Some refs:

* Original Paper: [Attention is all you need](https://arxiv.org/pdf/1706.03762.pdf);

* [The Illustrated Transformer - Jay Alammar.](http://jalammar.github.io/illustrated-transformer/)


Disclaimers:

We will train in a standard way a Transformer Model. This is not BERT, which is a collection of Transformer layers. BERT is trained according to the Masked Language Model (MLM) paradigm.

In [153]:
%%capture
!pip install torchtext==0.6.0 torchdata

# After the installation, restart the session

In [154]:
import torch
import torch.nn as nn
import torch.nn.functional as F


#from torchtext.legacy import data, datasets

from torchtext import data, datasets
from torchtext import vocab
import numpy as np
import random, tqdm, sys, math, gzip
from torchsummary import summary

from torch.utils.tensorboard import SummaryWriter

In [155]:
use_cuda = torch.cuda.is_available()
#device = torch.device("cuda" if use_cuda else "cpu")
device="cpu"

In [156]:
!nvidia-smi

Sat Jun  1 09:36:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0              30W /  70W |   1241MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Multi-head Attention Mechanism, step by step

First, let's set some hyperparameters. To keep it simple we choose small size hyperparameters.

In [157]:
emb = 128 # embedding dimension (BERT like models 768)
h = 8 # number of heads (BERT has 12 heads)

batch_size = 4
sentence_length = 21 # Context Length, 512 for BERT

Some fake random data with proper dimensions

In [158]:
x = torch.rand(batch_size, 21, emb)

b, t, e = x.size()

In [159]:
x.size()

torch.Size([4, 21, 128])

Instantiate linear transformations for query, key and values. Each transformation will act on the input vector x.

In [160]:
tokeys    = nn.Linear(emb, emb, bias=False) # W_key
toqueries = nn.Linear(emb, emb, bias=False) # W_query
tovalues  = nn.Linear(emb, emb, bias=False) # W_value

Generate queries, keys and values. We first compute the k/q/v's on the whole embedding vectors, and then split into the different heads.

In [161]:
keys    = tokeys(x) # W_key x
queries = toqueries(x)
values  = tovalues(x)

In [162]:
print(keys.size())

torch.Size([4, 21, 128])


Implement now multi-head attention (the ligther version), splitting into the different heads.

In [163]:
s = e // h # 128 / 8

keys    = keys.view(b, t, h, s)
queries = queries.view(b, t, h, s)
values  = values.view(b, t, h, s)

print(keys.size())

torch.Size([4, 21, 8, 16])


In [164]:
keys.transpose(1, 2).size()

torch.Size([4, 8, 21, 16])

In [165]:
keys.transpose(1, 2).contiguous().view(b * h, t, s).size()

torch.Size([32, 21, 16])

We need now to compute the dot products. This is the same operation for every head, so we fold the heads into the batch dimension.

In [166]:
keys = keys.transpose(1, 2).contiguous().view(b * h, t, s)
queries = queries.transpose(1, 2).contiguous().view(b * h, t, s)
values = values.transpose(1, 2).contiguous().view(b * h, t, s)

# contiguous():  it actually makes a copy of the tensor such that the order of
# its elements in memory is the same as if it had been created from scratch with the same data.
# transpose(1, 2) doesn't generate a new tensor with a new layout, it just
# modifies meta information in the Tensor object so that the offset and stride describe the desired new shape.
# https://discuss.pytorch.org/t/contigious-vs-non-contigious-tensor/30107

keys.size()

torch.Size([32, 21, 16])

Perform dot products

In [167]:
print(queries.size())

print(keys.transpose(1, 2).size())

#Let's compute the attention matrix
attn_scores = torch.bmm(queries, keys.transpose(1, 2)).size()  # batch matrix-matrix product

print(attn_scores)

torch.Size([32, 21, 16])
torch.Size([32, 16, 21])
torch.Size([32, 21, 21])


💡 Note: `torch.bmm` and the symbol `@` are the same thing. You can check easily:

```
mat1 = torch.randn(10, 3, 4)
mat2 = torch.randn(10, 4, 5)
res_1 = torch.bmm(mat1, mat2)
res_2 = mat1 @ mat2
```
In particular:
* `torch.bmm`: Performs a batch matrix-matrix product of 3D tensors `mat1.size() = [b,n,m]` and `mat2.size() = [b,m,p]`, outputting `res.size() = [b,n,p]`. Here is the [documentation](https://pytorch.org/docs/stable/generated/torch.bmm.html#torch.bmm).
* symbol `@`: The matrix multiplication(s) are done between the last two dimensions. The remaining dimensions are broadcast and batched.




Just for completeness, below the implementation of the original multi-head attention (which is wide and computationally more intensive).

In [168]:
emb = 128
h = 8

x = torch.rand(4, 21, emb)

b, t, e = x.size()

tokeys    = nn.Linear(emb, emb * h, bias=False)
toqueries = nn.Linear(emb, emb * h, bias=False)
tovalues  = nn.Linear(emb, emb * h, bias=False)

keys    = tokeys(x)
queries = toqueries(x)
values  = tovalues(x)

print(keys.size())

keys    = keys.view(b, t, h, e)
queries = queries.view(b, t, h, e)
values  = values.view(b, t, h, e)

print(keys.size())


torch.Size([4, 21, 1024])
torch.Size([4, 21, 8, 128])


### Model Definition

Let us collect everything and define the self-attention class

In [169]:
class MHSelfAttention(nn.Module):
    """
    Multi-head self attention.
    """

    def __init__(self, emb, heads=8):
        """
        :param emb:
        :param heads:
        :param mask:
        """
        super().__init__()

        assert emb % heads == 0, f'Embedding dimension ({emb}) should be divisible by nr. of heads ({heads})'

        self.emb = emb
        self.heads = heads

        #s = emb // heads
        # - We will break the embedding into `heads` chunks and feed each to a different attention head

        self.tokeys    = nn.Linear(emb, emb, bias=False) # W_key
        self.toqueries = nn.Linear(emb, emb, bias=False) # W_query
        self.tovalues  = nn.Linear(emb, emb, bias=False) # W_value

        self.unifyheads = nn.Linear(emb, emb)

    def forward(self, x):

        b, t, e = x.size()
        h = self.heads
        assert e == self.emb, f'Input embedding dim ({e}) should match layer embedding dim ({self.emb})'

        s = e // h

        # We first compute the k/q/v's on the whole embedding vectors, and then split into the different heads.

        keys    = self.tokeys(x)
        queries = self.toqueries(x)
        values  = self.tovalues(x)

        # Split into the different heads.

        keys    = keys.view(b, t, h, s)
        queries = queries.view(b, t, h, s)
        values  = values.view(b, t, h, s)

        # Compute scaled dot-product self-attention

        # Fold heads into the batch dimension
        # When you call contiguous(), it actually makes a copy of the tensor
        # such that the order of its elements in memory is the same as if it had been created from scratch with the same data.
        keys = keys.transpose(1, 2).contiguous().view(b * h, t, s)
        queries = queries.transpose(1, 2).contiguous().view(b * h, t, s)
        values = values.transpose(1, 2).contiguous().view(b * h, t, s)

        queries = queries / (e ** (1/4))
        keys    = keys / (e ** (1/4))
        # Instead of dividing the dot products by sqrt(e), we scale the keys and values.
        # This should be more memory efficient

        # Get dot product of queries and keys, and scale.

        attn_scores = torch.bmm(queries, keys.transpose(1, 2))

        assert attn_scores.size() == (b * h, t, t)

        attn_weights = F.softmax(attn_scores, dim=2) # Dot now has row-wise self-attention probabilities

        # apply the self attention to the values
        out = torch.bmm(attn_weights, values).view(b, h, t, s)

        # swap h, t back, unify heads
        out = out.transpose(1, 2).contiguous().view(b, t, s * h)

        return self.unifyheads(out)

A Transformer Block is based on self-attention (and Layer Normalization, Residual Connections)

In [170]:
class TransformerBlock(nn.Module):

    def __init__(self, emb, heads, mask, seq_length, ff_hidden_mult=4, dropout=0.0, pos_embedding=None):
        super().__init__()

        self.mhattention = MHSelfAttention(emb, heads=heads)

        self.norm1 = nn.LayerNorm(emb)
        self.norm2 = nn.LayerNorm(emb)

        self.ff = nn.Sequential(
            nn.Linear(emb, ff_hidden_mult * emb),
            nn.ReLU(),
            nn.Linear(ff_hidden_mult * emb, emb)
        )

        self.do = nn.Dropout(dropout)

    def forward(self, x):

        attended = self.mhattention(x)

        x = self.norm1(attended + x) #residual

        x = self.do(x)

        fedforward = self.ff(x)

        x = self.norm2(fedforward + x) #residual

        x = self.do(x)

        return x

Let's build a Transformers (a stack of Transformers Blocks) and adapt it for a binary classification task. Its `depth` defines the number of Transformers Blocks

In [171]:
class CTransformer(nn.Module):

    def __init__(self, emb, heads, depth, seq_length, num_tokens, num_classes, max_pool=True, dropout=0.0):
        """
        :param emb: Embedding dimension
        :param heads: nr. of attention heads
        :param depth: Number of transformer blocks
        :param seq_length: Expected maximum sequence length
        :param num_tokens: Number of tokens (usually words) in the vocabulary
        :param num_classes: Number of classes.
        :param max_pool: If true, use global max pooling in the last layer. If false, use global
                         average pooling.
        """
        super().__init__()

        self.num_tokens, self.max_pool = num_tokens, max_pool

        # Token embedding
        self.token_embedding = nn.Embedding(embedding_dim=emb, num_embeddings=num_tokens)
        # Position embedding
        self.pos_embedding = nn.Embedding(embedding_dim=emb, num_embeddings=seq_length)

        tblocks = []
        for i in range(depth):
            tblocks.append(
                TransformerBlock(emb=emb, heads=heads, seq_length=seq_length, mask=False, dropout=dropout))

        self.tblocks = nn.Sequential(*tblocks)

        self.toprobs = nn.Linear(emb, num_classes)

        self.do = nn.Dropout(dropout)

    def forward(self, x):
        """
        :param x: A batch by sequence length integer tensor of token indices.
        :return: predicted log-probability vectors for each token based on the preceding tokens.
        """
        tokens = self.token_embedding(x)
        b, t, e = tokens.size()

        positions = self.pos_embedding(torch.arange(t, device=device))[None, :, :].expand(b, t, e)
        x = tokens + positions
        x = self.do(x)

        x = self.tblocks(x)

        x = x.max(dim=1)[0] if self.max_pool else x.mean(dim=1) # pool over the time dimension

        x = self.toprobs(x)

        return F.log_softmax(x, dim=1) # nn.softmax()

In [172]:
torch.arange(21)
# [0, 0, ..., 0]
# [1, 1, ..., 1]
# ...
# [20, 20, ..., 20]

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20])

### Data Preparation

One of the main concepts of TorchText is the `Field`. These define how your data should be processed. In our sentiment classification task the data consists of both the raw string of the review and the sentiment, either "pos" or "neg".

The parameters of a `Field` specify how the data should be processed.

We use the `TEXT` field to define how the review should be processed, and the `LABEL` field to process the sentiment.


In [173]:
import os
import sys
import numpy as np
import pandas as pd
import random

def get_IMDB_from_torchtext():
    SEED=42
    VOC_SIZE = 50000
    BATCH_SIZE = 4
    TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
    LABEL = data.LabelField(sequential=False, dtype = torch.float, batch_first=True)
    train, test = datasets.IMDB.splits(TEXT, LABEL)
    #TEXT.build_vocab(train, max_size=VOC_SIZE - 2)
    #LABEL.build_vocab(train)
    #train_iter, test_iter = data.Iterator.splits((train, test), batch_size=1, device = 'cpu')
    #train_iter, test_iter  = data.BucketIterator.splits((train, test), batch_size=BATCH_SIZE, device='cpu')
    #train_iter, valid_iter = train_iter.split(random_state = random.seed(SEED), split_ratio=0.8)

    labels, reviews = [], []
    for line in train.examples:
        #print(item.label, item.text )
        label=vars(line)['label']
        review=''.join(str(var)+" " for var in vars(line)['text'])
        assert label in ('pos', 'neg')
        #print(label)
        labels.append(label)
        reviews.append(review)
    df_train = pd.DataFrame({'sentiment': labels, 'review': reviews})
    df_train['sentiment'] = df_train['sentiment'].map({'pos': 1, 'neg': 0})
    print('original df_train.shape: ', df_train.shape)
    #df_train = df_train.drop_duplicates()
    #print('after drop_duplicates, df_train.shape: ', df_train.shape)
    labels, reviews = [], []
    for line in test.examples:
        label=vars(line)['label']
        review=vars(line)['text']
        assert label in ('pos', 'neg')
        labels.append(label)
        reviews.append(review)
    df_test = pd.DataFrame({'sentiment': labels, 'review': reviews})
    df_test['sentiment'] = df_test['sentiment'].map({'pos': 1, 'neg': 0})
    print('original df_test.shape: ', df_test.shape)
    #df_test = df_test.drop_duplicates()
    #print('after drop_duplicates, df_test.shape: ', df_test.shape)

    return df_train, df_test




df_train, df_test = get_IMDB_from_torchtext()


original df_train.shape:  (25000, 2)
original df_test.shape:  (25000, 2)


In [174]:

df_train.to_csv('./imdb_train.csv', index=True)
df_test.to_csv('./imdb_test.csv', index=True)

In [175]:
df_train.head(5)

,sentiment,review
0,1,"the movie held my interest, mainly because dia..."
1,1,the sopranos (1999-2007)<br /><br />number 1 -...
2,1,one of the funnest mario's i've ever played. t...
3,1,i was lucky enough to get a dvd copy of this m...
4,1,a frolics of youth short subject.<br /><br />a...


In [176]:
print(vars(train_data[0]))

{'sentiment': '1', 'review': ['the', 'movie', 'held', 'my', 'interest,', 'mainly', 'because', 'dianne', 'keaton', 'is', 'my', 'favorite', 'actress.', 'i', 'disagree', 'with', 'some', 'of', 'the', 'other', 'posts', 'on', 'the', 'grounds', 'that', 'the', 'plot', 'was', 'not', 'convoluted.', 'i', 'had', 'no', 'trouble', 'following', 'it', '(maybe', 'some', 'people', 'had', 'too', 'much', 'eggnog', 'the', 'night', 'before).', 'the', 'movie', 'was', 'very', 'sad', 'and', 'touching', 'as', 'well.', 'what', 'more', 'do', 'you', 'want?', 'alexa', 'davalos', 'is', 'a', 'fine', 'new', 'talent', '(beautiful', 'too),', 'and', 'tom', 'everett', 'scott', 'does', 'an', 'excellent', 'job', 'with', 'his', 'part', 'as', 'well.', 'the', 'relationship', 'of', 'the', 'mother', 'and', 'daughter', 'may', 'have', 'been', 'a', 'bit', 'unrealistic,', 'but', 'the', 'behavior', 'of', 'the', 'young', 'people', 'in', 'the', 'movie', 'was', 'not.', 'it', 'was', 'tragically', 'sad', 'but', 'enlightening.', 'it', 'sur

In [177]:
TEXT=data.Field(batch_first=True) # If no tokenize argument is passed, the default is simply splitting the string on spaces.
LABEL=data.LabelField(sequential=False, dtype = torch.float, batch_first=True,use_vocab=False)

NUM_CLS = 2
BATCH_SIZE = 4
MAX_LENGTH = 256 #512
EMB_SIZE = 128
HEADS = 8
DEPTH = 3 #Number of self-attention layer
VOC_SIZE = 50000

LR_RATE = 0.0001
WARMUP = 10000

In [178]:
fields = [(None, None),('sentiment',LABEL),('review',TEXT)]

train_data, test_data = data.TabularDataset.splits(
                                        path = './',
                                        train = 'imdb_train.csv',
                                        test = 'imdb_test.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)



In [179]:
#tbw = SummaryWriter(log_dir='./logs') # Tensorboard logging

#train, test = datasets.IMDB.splits(TEXT, LABEL)

In [180]:
#TEXT = data.Field(tokenize='spacy', batch_first=True)

TEXT.build_vocab(train_data, max_size = VOC_SIZE-2)
LABEL.build_vocab(train_data)
#TEXT.build_vocab(train, max_size=VOC_SIZE - 2)
#LABEL.build_vocab(train)

In [181]:
TEXT.vocab.freqs.most_common(10)

[('the', 322198),
 ('a', 159953),
 ('and', 158572),
 ('of', 144462),
 ('to', 133967),
 ('is', 104171),
 ('in', 90527),
 ('i', 70480),
 ('this', 69714),
 ('that', 66292)]

In [182]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device='cpu'

BATCH_SIZE = 4

train_iter, test_iter = data.BucketIterator.splits(
    (train_data, test_data),
    sort_key = lambda x: x.sentiment, #sort by s attribute (sentiment)
    batch_size=BATCH_SIZE,
    device=device)

In [183]:
# batch size = 4
for batch in train_iter:
    print("batch.sentiment.size:")
    print(batch.sentiment.size())
    print("batch.review:")
    print(batch.review.size())

    break

batch.sentiment.size:
torch.Size([4])
batch.review:
torch.Size([4, 570])


In [184]:
print(f'- nr. of training examples {len(train_iter)}')
print(f'- nr. of test examples {len(test_iter)}')

- nr. of training examples 6250
- nr. of test examples 6250


In [185]:
# create the model
model = CTransformer(emb=EMB_SIZE, heads=HEADS, depth=DEPTH, seq_length=MAX_LENGTH, num_tokens=VOC_SIZE, num_classes=NUM_CLS, max_pool="store_true", dropout=0.2)
model.to(device)

CTransformer(
  (token_embedding): Embedding(50000, 128)
  (pos_embedding): Embedding(256, 128)
  (tblocks): Sequential(
    (0): TransformerBlock(
      (mhattention): MHSelfAttention(
        (tokeys): Linear(in_features=128, out_features=128, bias=False)
        (toqueries): Linear(in_features=128, out_features=128, bias=False)
        (tovalues): Linear(in_features=128, out_features=128, bias=False)
        (unifyheads): Linear(in_features=128, out_features=128, bias=True)
      )
      (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ff): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): ReLU()
        (2): Linear(in_features=512, out_features=128, bias=True)
      )
      (do): Dropout(p=0.2, inplace=False)
    )
    (1): TransformerBlock(
      (mhattention): MHSelfAttention(
        (tokeys): Linear(in_features=128, out_features=128, bias=False)
   

In [186]:
opt = torch.optim.Adam(lr=LR_RATE, params=model.parameters())
sch = torch.optim.lr_scheduler.LambdaLR(opt, lambda i: min(i / (WARMUP / BATCH_SIZE), 1.0))

In [ ]:
NUM_EPOCHS = 4

# training loop
seen = 0
for e in range(NUM_EPOCHS):

    print(f'\n epoch {e}')
    model.train()

    for batch in tqdm.tqdm(train_iter):

        opt.zero_grad()

        input = batch.review
        input = input.to(device)
        label = batch.sentiment.to(torch.long)
        label = label.to(device)

        if input.size(1) > MAX_LENGTH:
            input = input[:, :MAX_LENGTH]


        out = model(input)
        loss = F.nll_loss(out, label)
        # loss = CrossEntropy(out, label)

        loss.backward()

        # clip gradients
        # Performs gradient clipping. It is used to mitigate the problem of exploding gradients.
        # - If the total gradient vector has a length > 1, we clip it back down to 1.
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        opt.step()
        sch.step()

        seen += input.size(0)
        tbw.add_scalar('classification/train-loss', float(loss.item()), seen)

    with torch.no_grad():

        model.eval()
        tot, cor= 0.0, 0.0

        for batch in test_iter:

            input = batch.review
            input = input.to(device)
            label = batch.sentiment.to(torch.long)
            label = label.to(device)
            if input.size(1) > MAX_LENGTH:
                input = input[:, :MAX_LENGTH]
            out = model(input).argmax(dim=1)

            tot += float(input.size(0))
            cor += float((label == out).sum().item())

        acc = cor / tot
        print(f'-- test accuracy {acc:.3}')
        tbw.add_scalar('classification/test-loss', float(loss.item()), e)



 epoch 0


 81%|████████  | 5071/6250 [07:25<01:46, 11.04it/s]

In [ ]:
#TODO


model_inference = model.load_weigths('path')

model_inference("I bought that book and I enjoyed the readings")

# 0 o 1